In [ ]:
!pip install vecstack

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for vecstack: filename=vecstack-0.4.0-py3-none-any.whl size=19877 sha256=e816c81283f2d7f9179fdcfb942727ce711131ebcf803a4b918351197d7e73eb
  Stored in directory: /root/.cache/pip/wheels/28/fe/0c/fe8e43660e3316d7ce204e59a79a72246c0ae9b6c5c79841c8
Successfully built vecstack


In [ ]:
from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from vecstack import stacking
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import SMOTE 
from sklearn.svm import SVC
from collections import Counter
from sklearn._config import get_config, set_config
set_config(print_changed_only=False)

In [ ]:
trainfile = r'/gdrive/My Drive/RevisedHomesiteTrain1.csv'
train_data = pd.read_csv(trainfile)

testfile = r'/gdrive/My Drive/RevisedHomesiteTest1.csv'
test_data = pd.read_csv(testfile)

print(train_data.shape)
print(test_data.shape)

(65000, 596)
(173836, 596)


In [ ]:
test_data.isnull().sum()

CoverageField11A        0
CoverageField11B        0
CoverageField1A         0
CoverageField1B         0
CoverageField2A         0
                       ..
GeographicField64_CA    0
GeographicField64_IL    0
GeographicField64_NJ    0
GeographicField64_TX    0
GeographicField64       0
Length: 596, dtype: int64

In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65000 entries, 0 to 64999
Columns: 596 entries, CoverageField11A to QuoteConversion_Flag
dtypes: float64(5), int64(591)
memory usage: 295.6 MB


In [ ]:
train_data.describe()

,CoverageField11A,CoverageField11B,CoverageField1A,CoverageField1B,CoverageField2A,CoverageField2B,CoverageField3A,CoverageField3B,CoverageField4A,CoverageField4B,...,PropertyField38_N,PropertyField38_Y,GeographicField63_,GeographicField63_N,GeographicField63_Y,GeographicField64_CA,GeographicField64_IL,GeographicField64_NJ,GeographicField64_TX,QuoteConversion_Flag
count,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,...,65000.000000,65000.000000,65000.0,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000
mean,7.798892,12.954892,9.014585,12.995831,9.060000,13.017308,9.064538,13.017877,8.973785,13.011031,...,0.974969,0.020292,0.0,0.976262,0.023738,0.361862,0.120908,0.267631,0.249600,0.188646
std,5.304856,7.259023,5.586114,7.217093,5.568791,7.207118,5.588694,7.212825,5.564675,7.203325,...,0.156220,0.140999,0.0,0.152234,0.152234,0.480543,0.326022,0.442727,0.432785,0.391230
min,-1.000000,-1.000000,-1.000000,-1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.000000,7.000000,5.000000,7.000000,5.000000,7.000000,5.000000,7.000000,5.000000,7.000000,...,1.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,6.000000,13.000000,8.000000,13.000000,8.000000,13.000000,8.000000,13.000000,8.000000,13.000000,...,1.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,10.000000,19.000000,12.000000,19.000000,12.000000,19.000000,12.000000,19.000000,12.000000,19.000000,...,1.000000,0.000000,0.0,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000
max,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,...,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
train_data["QuoteConversion_Flag"].value_counts()

0    52738
1    12262
Name: QuoteConversion_Flag, dtype: int64

In [ ]:
Xtrain = train_data.drop(["QuoteConversion_Flag"],axis=1)
Ytrain= pd.DataFrame(train_data["QuoteConversion_Flag"])

In [ ]:
test_data = test_data.drop(["GeographicField64"],axis=1)

In [ ]:
smote=SMOTE(sampling_strategy="minority")

In [ ]:
X_sm,Y_sm = smote.fit_resample(Xtrain,Ytrain)

In [ ]:
Y_sm["QuoteConversion_Flag"].value_counts()

0    52738
1    52738
Name: QuoteConversion_Flag, dtype: int64

In [ ]:
X_train_sm, X_test_sm, Y_train_sm, Y_test_sm = train_test_split(X_sm,Y_sm, test_size = .30, random_state = 1,stratify=Y_sm)

In [ ]:
#DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(X_train_sm,Y_train_sm)
dt_predict = dt.predict(X_test_sm)

In [ ]:
print("accuracy Score (training) for Decision Tree:{0:6f}".format(dt.score(X_train_sm,Y_train_sm)))
print("Confusion Matrix for Decision Tree")
print(confusion_matrix(Y_test_sm,dt_predict))
print("Accuracy :", metrics.accuracy_score(Y_test_sm,dt_predict))
print("Classification report:", classification_report(Y_test_sm,dt_predict))

accuracy Score (training) for Decision Tree:1.000000
Confusion Matrix for Decision Tree
[[14496  1325]
 [ 1226 14596]]
Accuracy : 0.919381853806529
Classification report:               precision    recall  f1-score   support

           0       0.92      0.92      0.92     15821
           1       0.92      0.92      0.92     15822

    accuracy                           0.92     31643
   macro avg       0.92      0.92      0.92     31643
weighted avg       0.92      0.92      0.92     31643



In [ ]:
#RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train_sm,Y_train_sm)
rf_predict = rf.predict(X_test_sm)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
print("accuracy Score (training) for Random Forest:{0:6f}".format(rf.score(X_train_sm,Y_train_sm)))
print("Confusion Matrix for Random Forest")
print(confusion_matrix(Y_test_sm,rf_predict))
print("Accuracy :", metrics.accuracy_score(Y_test_sm,rf_predict))
print("Classification report:", classification_report(Y_test_sm,rf_predict))

accuracy Score (training) for Random Forest:1.000000
Confusion Matrix for Random Forest
[[15234   587]
 [ 1321 14501]]
Accuracy : 0.9397023038270708
Classification report:               precision    recall  f1-score   support

           0       0.92      0.96      0.94     15821
           1       0.96      0.92      0.94     15822

    accuracy                           0.94     31643
   macro avg       0.94      0.94      0.94     31643
weighted avg       0.94      0.94      0.94     31643



In [ ]:
#SVM Classifier
svm = SVC() 
svm.fit(X_train_sm,Y_train_sm) 
svm_predict=svm.predict(X_test_sm)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
print("accuracy Score (training) for SVM:{0:6f}".format(svm.score(X_train_sm,Y_train_sm)))
print("Confusion Matrix for SVM")
print(confusion_matrix(Y_test_sm,svm_predict))
print("Accuracy :", metrics.accuracy_score(Y_test_sm,svm_predict))
print("Classification report:", classification_report(Y_test_sm,svm_predict))

accuracy Score (training) for SVM:0.606883
Confusion Matrix for SVM
[[ 7803  8018]
 [ 4431 11391]]
Accuracy : 0.6065796542679266
Classification report:               precision    recall  f1-score   support

           0       0.64      0.49      0.56     15821
           1       0.59      0.72      0.65     15822

    accuracy                           0.61     31643
   macro avg       0.61      0.61      0.60     31643
weighted avg       0.61      0.61      0.60     31643



In [ ]:
#K-Nearest Neighbours Classifier
neigh = KNeighborsClassifier()
neigh.fit(X_train_sm,Y_train_sm)
neigh_predict=neigh.predict(X_test_sm)

/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [ ]:
print("accuracy Score (training) for KNN :{0:6f}".format(neigh.score(X_train_sm,Y_train_sm)))
print("Confusion Matrix for KNN")
print(confusion_matrix(Y_test_sm,neigh_predict))
print("Accuracy :", metrics.accuracy_score(Y_test_sm,neigh_predict))
print("Classification report:", classification_report(Y_test_sm,neigh_predict))

accuracy Score (training) for KNN :0.820609
Confusion Matrix for KNN
[[ 9384  6437]
 [ 2472 13350]]
Accuracy : 0.718452738362355
Classification report:               precision    recall  f1-score   support

           0       0.79      0.59      0.68     15821
           1       0.67      0.84      0.75     15822

    accuracy                           0.72     31643
   macro avg       0.73      0.72      0.71     31643
weighted avg       0.73      0.72      0.71     31643



In [ ]:
#MultiLayerPerceptron
mlp = MLPClassifier()
mlp.fit(X_train_sm,Y_train_sm)
mlp_predict=mlp.predict(X_test_sm)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
print("accuracy Score (training) for MLP :{0:6f}".format(mlp.score(X_train_sm,Y_train_sm)))
print("Confusion Matrix for MLP")
print(confusion_matrix(Y_test_sm,mlp_predict))
print("Accuracy :", metrics.accuracy_score(Y_test_sm,mlp_predict))
print("Classification report:", classification_report(Y_test_sm,mlp_predict))

accuracy Score (training) for MLP :0.779394
Confusion Matrix for MLP
[[ 9290  6531]
 [  520 15302]]
Accuracy : 0.7771703062288657
Classification report:               precision    recall  f1-score   support

           0       0.95      0.59      0.72     15821
           1       0.70      0.97      0.81     15822

    accuracy                           0.78     31643
   macro avg       0.82      0.78      0.77     31643
weighted avg       0.82      0.78      0.77     31643



In [ ]:
model = [DecisionTreeClassifier(),RandomForestClassifier(),SVC(), KNeighborsClassifier(),MLPClassifier()]

In [ ]:
S_Train, S_Test = stacking(model,                   
                           X_sm, Y_sm, test_data,   
                           regression=False, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric=accuracy_score, 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [5]

model  0:     [DecisionTreeClassifier]
    fold  0:  [0.91838902]
    fold  1:  [0.91876825]
    fold  2:  [0.91929918]
    fold  3:  [0.92165042]
    ----
    MEAN:     [0.91952672] + [0.00126802]
    FULL:     [0.91952672]

model  1:     [RandomForestClassifier]
    fold  0:  [0.94049831]
    fold  1:  [0.94027077]
    fold  2:  [0.94030870]
    fold  3:  [0.94068793]
    ----
    MEAN:     [0.94044143] + [0.00016639]
    FULL:     [0.94044143]

model  2:     [SVC]
    fold  0:  [0.60673518]
